In [2]:
# Dependencies
import pandas as pd 
import numpy as np 

from time import time

import warnings
warnings.filterwarnings("ignore")

In [12]:
# Read in data from S3 Buckets
# url = "https://job-postings-dataviz.s3.amazonaws.com/fake_jobs_clean.csv"
# df = pd.read_csv(url, sep=",",  encoding = "UTF-8" )
# Read data from ../assets/data/
# df = pd.read_csv("../assets/data/fake_jobs_clean.csv",sep=",",  encoding = "UTF-8")
# Read data imported to the drive to use in Colab
df = pd.read_csv("/content/fake_jobs_clean.csv", encoding = "UTF-8")
df.head()

,job_id,city,state/province,country,title,department,industry,function,salary_range,salary_provided,company_profile,description,requirements,benefits,benefits_provided,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,fraudulent
0,1,New York,NY,US,Marketing Intern,Marketing,NaN,Marketing,NaN,0,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,0,1,0,Other,Internship,NaN,0
1,2,Auckland,,NZ,Customer Service - Cloud Video Production,Success,Marketing and Advertising,Customer Service,NaN,0,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,1,0,1,0,Full-time,Not Applicable,NaN,0
2,3,Wever,IA,US,Commissioning Machinery Assistant (CMA),NaN,NaN,NaN,NaN,0,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,0,1,0,NaN,NaN,NaN,0
3,4,Washington,DC,US,Account Executive - Washington DC,Sales,Computer Software,Sales,NaN,0,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,1,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,0
4,5,Fort Worth,FL,US,Bill Review Manager,NaN,Hospital & Health Care,Health Care Provider,NaN,0,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,1,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,0


In [13]:
# Focus only on the job description. Create a dataFrame with label "fraudulent" and "description"
descrip_df =df [['fraudulent','description']]
descrip_df.head()

,fraudulent,description
0,0,"Food52, a fast-growing, James Beard Award-winn..."
1,0,Organised - Focused - Vibrant - Awesome!Do you...
2,0,"Our client, located in Houston, is actively se..."
3,0,THE COMPANY: ESRI – Environmental Systems Rese...
4,0,JOB TITLE: Itemization Review ManagerLOCATION:...


In [17]:
# Drop NaN values but dropping duplicates is questionable
print(len(descrip_df))
descrip_df = descrip_df.dropna()
print(len(descrip_df))

17879
17879


In [16]:
# We are considering that these duplicates are real data and contribute to both classes
# descripduplicates_df = descrip_df.drop_duplicates()
# print(descripduplicates_df.count())

fraudulent     14802
description    14802
dtype: int64


In [18]:
# Make sure number of real vs fraudulent are more balanced (1:5)
df_fake= descrip_df[descrip_df['fraudulent'] == 1] 
df_real = descrip_df[descrip_df['fraudulent'] == 0] 
# Count how many Real vs Fraudulent postings
n_f=len(df_fake)
n_n=len(df_real)
print(n_f)
print(n_n)

865
17014


Create a more balanced data set (1:5 fake to real) It is still understimating the real jobs

In [20]:
df_new= df_real.sample  ( 5*n_f  , random_state=580)
df_unders = df_new.append(df_fake)
print(len(df_new))
print(len(df_unders))

4325
5190


In [21]:
# Shuffle the understimate dataframe so "fraudulent" postings are more evenly distributed 
df_underst = df_unders.sample(len(df_unders), random_state=580) 
# df_underst.head(20)
X= df_underst.iloc[:,1].values
y=df_underst.iloc[:,0].values

### Feature Transformations (Term Frequency times inverse document frequency)


In [25]:
# feature extraction using "bag of words" model. Create feature vectors, remove stop words and built inverse frequncies
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# import nltk
# nltk.download('stopwords')

from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

count_vectorizer = CountVectorizer( stop_words=stopwords.words('english'))
tfidfconverter = TfidfTransformer()

In [26]:
X_c = count_vectorizer.fit_transform(X)
X_tfidf = tfidfconverter.fit_transform(X_c).toarray()

Create a Naive Bayes Model  

In [27]:
# Break the data in train and test datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, random_state=580, stratify=y)

Multinomial Naive Bayes

In [29]:
# Create a Naive Bayes model and fit training data
from sklearn.naive_bayes import ComplementNB

c_nb = ComplementNB(alpha= 0.8)

t1_start = time()
predictor = c_nb.fit(X_train,y_train)
t1_stop = time() 
print("Elapsed time:", t1_stop, t1_start)   
print(f'Elapsed time during the whole program in seconds:',t1_stop-t1_start)

Elapsed time: 1616005883.4763215 1616005882.9537206
Elapsed time during the whole program in seconds: 0.5226008892059326


In [30]:
# Tranform the model with the testing data
test_results = predictor.predict(X_test)

In [31]:
from sklearn.metrics import classification_report

print(classification_report(y_test, test_results,
                            target_names= ["real", "fake"]))

              precision    recall  f1-score   support

        real       0.87      0.99      0.93      1082
        fake       0.84      0.26      0.40       216

    accuracy                           0.87      1298
   macro avg       0.85      0.63      0.66      1298
weighted avg       0.87      0.87      0.84      1298



Save the model 

In [ ]:
# save the model
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Save to the model to the local drive

from sklearn.externals import joblib

filename = '/content/drive/My Drive/NB_under.h5'
joblib.dump(predictor, filename)

['/content/drive/My Drive/NB_under.h5']

In [ ]:
# Load the model
loaded_model= joblib.load(filename)
predictionsl = loaded_model.predict(X_test)


In [32]:
print(classification_report(y_test, test_results,
                            target_names= ["real", "fake"]))

              precision    recall  f1-score   support

        real       0.87      0.99      0.93      1082
        fake       0.84      0.26      0.40       216

    accuracy                           0.87      1298
   macro avg       0.85      0.63      0.66      1298
weighted avg       0.87      0.87      0.84      1298

